<a href="https://colab.research.google.com/github/Nickguild1993/Py_EDA_walks/blob/main/Occupation_PivotTable_CrossTabs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Grouping, aggregating etc on Occupation dataset

In [2]:
# import libraries

import pandas as pd
import numpy as np
import random as random

In [3]:
# get dataset from github raw url
# | delim
df = pd.read_csv("https://raw.githubusercontent.com/justmarkham/DAT8/master/data/u.user", delimiter="|")
df.head(5)

,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [4]:
# the usual 
print(df.shape)
print("-------------------------")
print(df.isnull().sum())
print("-------------------------")
print(df["occupation"].value_counts()[:5])

(943, 5)
-------------------------
user_id       0
age           0
gender        0
occupation    0
zip_code      0
dtype: int64
-------------------------
student          196
other            105
educator          95
administrator     79
engineer          67
Name: occupation, dtype: int64


In [5]:
df = df.set_index("user_id")

Get mean age per occupation

In [6]:
mean_age = df.groupby("occupation").agg({"age": "mean"})
print(mean_age.sort_values(by = "age", ascending = False))


                     age
occupation              
retired        63.071429
doctor         43.571429
educator       42.010526
healthcare     41.562500
librarian      40.000000
administrator  38.746835
executive      38.718750
marketing      37.615385
lawyer         36.750000
engineer       36.388060
writer         36.311111
salesman       35.666667
scientist      35.548387
other          34.523810
technician     33.148148
programmer     33.121212
homemaker      32.571429
artist         31.392857
entertainment  29.222222
none           26.555556
student        22.081633


Get Male / Female ratio per occupation and sort by greatest to least 

Not quite sure how to do this but here we go

In [7]:
cross = pd.crosstab(index=df["occupation"], columns = df["gender"])
cross.sort_values(by="M", ascending = False)[:5]

gender,F,M
occupation,,
student,60,136
other,36,69
educator,26,69
engineer,2,65
programmer,6,60


Calculate the min and max ages for each occupation

In [8]:
ages = df.groupby("occupation").agg({"age": ["min", "max", "mean"]})
print(ages.columns)

MultiIndex([('age',  'min'),
            ('age',  'max'),
            ('age', 'mean')],
           )


In [9]:
ages = ages.reset_index()
print(ages.columns)

MultiIndex([('occupation',     ''),
            (       'age',  'min'),
            (       'age',  'max'),
            (       'age', 'mean')],
           )


In [10]:
ages[:5]

occupation age               
                 min max       mean
0  administrator  21  70  38.746835
1         artist  19  48  31.392857
2         doctor  28  64  43.571429
3       educator  23  63  42.010526
4       engineer  22  70  36.388060

For each combination of occupation and gender, calculate the mean age

In [11]:
pivot_males = df.pivot_table(values=["age"], index = ["occupation", "gender"], aggfunc={"age": "mean"}, fill_value=0)
# shape is 41,1!!!!!
pivot_males.head(5)


age
occupation    gender           
administrator F       40.638889
              M       37.162791
artist        F       30.307692
              M       32.333333
doctor        M       43.571429

I like crosstab more than groupby here b/c
- it returns NaNs instead of skipping the values all together if absent
- less cumbersome

In [12]:
gender_group = df.groupby(["occupation", "gender"])["age"].mean().round(1)
gender_group = gender_group.reset_index(level=0)
gender_group = pd.DataFrame(gender_group)
gender_group.head(6)

,occupation,age
gender,,
F,administrator,40.6
M,administrator,37.2
F,artist,30.3
M,artist,32.3
M,doctor,43.6
F,educator,39.1


In [13]:


gender_cross = pd.crosstab(index = df["occupation"], 
                             columns = df["gender"],
                             values = df["age"],
                             aggfunc = np.mean).round(1)

gender_cross.head(5)

gender,F,M
occupation,,
administrator,40.6,37.2
artist,30.3,32.3
doctor,NaN,43.6
educator,39.1,43.1
engineer,29.5,36.6


Get percentage of women and men for each occupation

In [14]:
# (100 * gender_group["age"] / gender_group["age"].sum()).round(1)
# (gender_cross["F"] + gender_cross["M"]) / (gender_cross["F"])

# male percentage
male_percentage = (gender_cross["M"] / (gender_cross["F"] + gender_cross["M"]))

# female percentage 
female_percentage = (gender_cross["F"] / (gender_cross["F"] + gender_cross["M"]))

In [15]:
male_percentage

occupation
administrator    0.478149
artist           0.515974
doctor                NaN
educator         0.524331
engineer         0.553707
entertainment    0.483333
executive        0.464720
healthcare       0.532864
homemaker        0.402098
lawyer           0.478203
librarian        0.500000
marketing        0.504660
none             0.337568
other            0.489209
programmer       0.507645
retired          0.471698
salesman         0.588415
scientist        0.561920
student          0.521839
technician       0.464789
writer           0.484225
dtype: float64

In [16]:
new_df = df.copy()
new_df.head(1)

,age,gender,occupation,zip_code
user_id,,,,
1,24,M,technician,85711


In [17]:
# new_df.insert(3, "percentage_male", male_percentage)
# new_df.insert(4, "percentage_female", female_percentage)

percents_df = pd.DataFrame({"male_percentage" : male_percentage, "female_percentage": female_percentage})
percents_df.head(3)

,male_percentage,female_percentage
occupation,,
administrator,0.478149,0.521851
artist,0.515974,0.484026
doctor,NaN,NaN


replace NaNs and concatenate em

In [18]:
column_means = percents_df.mean()
column_means.head(3)

# df_percents = df_means.fillna(df_means)
# df_percents.head(4)

male_percentage      0.493267
female_percentage    0.506733
dtype: float64

In [19]:
percents_df= percents_df.fillna(column_means)
percents_df.head(3)

,male_percentage,female_percentage
occupation,,
administrator,0.478149,0.521851
artist,0.515974,0.484026
doctor,0.493267,0.506733


In [20]:
combo = [new_df, percents_df]

In [21]:
percents_df.info

<bound method DataFrame.info of                male_percentage  female_percentage
occupation                                       
administrator         0.478149           0.521851
artist                0.515974           0.484026
doctor                0.493267           0.506733
educator              0.524331           0.475669
engineer              0.553707           0.446293
entertainment         0.483333           0.516667
executive             0.464720           0.535280
healthcare            0.532864           0.467136
homemaker             0.402098           0.597902
lawyer                0.478203           0.521797
librarian             0.500000           0.500000
marketing             0.504660           0.495340
none                  0.337568           0.662432
other                 0.489209           0.510791
programmer            0.507645           0.492355
retired               0.471698           0.528302
salesman              0.588415           0.411585
scientist         

In [22]:
male_percents = percents_df["male_percentage"]

female_percents = percents_df["female_percentage"]

In [23]:
new_df = new_df.insert(4, "male_percentage", male_percents)

In [24]:
# new_df = new_df.insert(5, "female_percentage", female_percents)

AttributeError: ignored